In [2]:
%load_ext autoreload
%autoreload

import os
import re
import _pickle as cPickle
from collections import OrderedDict, defaultdict, Counter
import argparse
import multiprocessing
import math
import pdb
import random

import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.sparse import csr_matrix
import nltk
from nltk import word_tokenize

from data_structure import Instance
from utils import apply_parallel, get_tokens, get_group_df, filter_words

np.random.seed(1234)
random.seed(1234)

pd.set_option('display.max_rows', 2000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# configure

In [3]:
parser = argparse.ArgumentParser()

parser.add_argument('-seed', type=int, default=1234)
parser.add_argument('-n_reviews', type=int, default=8)
parser.add_argument('-item_min_reviews', type=int, default=8)
parser.add_argument('-filter_doc_l', type=int, default=50)
parser.add_argument('-filter_sent_l', type=int, default=40)
parser.add_argument('-min_tf', type=int, default=16)

parser.add_argument('-n_per_item', type=int, default=12)
parser.add_argument('-train_dir', type=str, default='data/yelp/train')
parser.add_argument('-val_dir', type=str, default='data/yelp/val')
parser.add_argument('-test_dir', type=str, default='data/yelp/test')
parser.add_argument('-ref_path', type=str, default='data/yelp/references.csv')

config = parser.parse_args('')
config.raw_path = os.path.join('data', 'yelp/yelp_raw_df.pkl')
config.output_path = os.path.join('data', 'yelp/yelp_recursum_df.pkl')

In [4]:
# special tokens
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data 

## load train raw data

In [5]:
def get_data_raw_df(data_paths):
    data_raw_dfs = []
    for data_path in data_paths:
        f = open(data_path, 'r')
        item_raw_df = pd.read_json(f)
        f.close()
        data_raw_dfs.append(item_raw_df)
    data_raw_df = pd.concat(data_raw_dfs)
    data_raw_df['tokens'] = data_raw_df['text'].apply(get_tokens)
    data_raw_df = data_raw_df[data_raw_df['tokens'].apply(lambda tokens: len(tokens) > 2)]
    return data_raw_df

In [6]:
get_data_paths = lambda data_dir: [os.path.join(data_dir, data_name) for data_name in os.listdir(data_dir) if not data_name == 'store-to-nreviews.json']
train_data_paths = get_data_paths(config.train_dir)
val_data_paths = get_data_paths(config.val_dir)
test_data_paths = get_data_paths(config.test_dir)

In [7]:
%%time
train_raw_df = apply_parallel(train_data_paths, num_split=32, map_func=get_data_raw_df).reset_index()
len(train_raw_df)

CPU times: user 1min 5s, sys: 16.6 s, total: 1min 22s
Wall time: 12min 59s


2008099

## load ref raw data

In [8]:
%%time
val_tmp_df = apply_parallel(val_data_paths, num_split=8, map_func=get_data_raw_df).reset_index()

CPU times: user 8.26 s, sys: 9.65 s, total: 17.9 s
Wall time: 1min 16s


In [9]:
%%time
test_tmp_df = apply_parallel(test_data_paths, num_split=8, map_func=get_data_raw_df).reset_index()

CPU times: user 2.99 s, sys: 3.61 s, total: 6.61 s
Wall time: 1min 8s


In [10]:
def get_ref_raw_df(ref_path, train_raw_df, val_tmp_df, test_tmp_df):
    def get_review_business_id_dict(ref_df, train_raw_df, val_tmp_df, test_tmp_df):
        ref_review_ids = []
        for _, row in ref_df.iterrows():
            ref_review_ids += [row['Input.original_review_%i_id' % i] for i in range(config.n_reviews)]
        ref_review_id_df = pd.DataFrame(ref_review_ids, columns=['review_id']) # only review_id in reference.csv
        concat_raw_df = pd.concat([train_raw_df, val_tmp_df, test_tmp_df])[['review_id', 'business_id', 'stars']] # filter review_id and business_id pair
        review_business_id_dict = {row.review_id: row.business_id for _, row in pd.merge(ref_review_id_df, concat_raw_df).iterrows()}
        review_stars_dict = {row.review_id: row.stars for _, row in pd.merge(ref_review_id_df, concat_raw_df).iterrows()}
        return review_business_id_dict, review_stars_dict
    
    ref_df = pd.read_csv(ref_path)
    ref_df['business_id_csv'] = ref_df.apply(lambda row: row['Input.business_id'] if row['Input.business_id'] != '#NAME?' else 'null_%i' % row.name, axis=1)
    review_business_id_dict, review_stars_dict = get_review_business_id_dict(ref_df, train_raw_df, val_tmp_df, test_tmp_df)
    
    ref_raw_dfs = []
    for index, row in ref_df.iterrows():
        business_id = None 
        texts, review_ids, stars = [], [], []
        summary = row['Answer.summary']
        for i in range(config.n_reviews):
            text = row['Input.original_review_%i' % i]
            review_id = row['Input.original_review_%i_id' % i]
            star = review_stars_dict[review_id] if review_id in review_stars_dict else 3
            
            texts.append(text)
            review_ids.append(review_id)
            stars.append(star)
            
            if review_id == '#NAME?': review_id = None
            if business_id is None and review_id in review_business_id_dict: business_id = review_business_id_dict[review_id] # get business_id from review_business_id_dict
            
        if business_id is None: business_id = row['business_id_csv'] # if all review_id not in review_business_id_dict, then business_id in csv is used
        for text, review_id, star in zip(texts, review_ids, stars):
            ref_raw_dfs.append({'business_id': business_id, 'text': text, 'review_id': review_id, 'summary': summary, 'stars': star})
        
    ref_raw_df = pd.DataFrame(ref_raw_dfs)
    ref_raw_df['tokens'] = ref_raw_df['text'].apply(get_tokens)
    return ref_raw_df

In [11]:
ref_raw_df = get_ref_raw_df(config.ref_path, train_raw_df, val_tmp_df, test_tmp_df)
assert len(ref_raw_df) == 200*config.n_reviews

# group data by business id

## group ref df

In [20]:
def get_ref_group_df(data_df):
    group_df = data_df.groupby('business_id').agg({
        'text': lambda text_list: ' </DOC> '.join(list(text_list)),
        'tokens': lambda token_idxs_list: [sent_idxs for token_idxs in list(token_idxs_list) for sent_idxs in token_idxs],
        'summary': lambda summary_series: list(summary_series)[0], # only first column for each business id
        'stars': lambda stars_list: list(stars_list)
    })
    group_df = group_df.reset_index()
    group_df['doc_l'] = group_df['tokens'].apply(lambda tokens: len(tokens))
    group_df['max_sent_l'] = group_df['tokens'].apply(lambda tokens: max([len(line) for line in tokens]))
    group_df['sent_l'] = group_df['tokens'].apply(lambda tokens: [len(line) for line in tokens])
    group_df['summary_tokens'] = group_df['summary'].apply(get_tokens)
    group_df['summary_doc_l'] = group_df['summary_tokens'].apply(lambda tokens: len(tokens))
    group_df['summary_max_sent_l'] = group_df['summary_tokens'].apply(lambda tokens: max([len(line) for line in tokens]))
    
    return group_df

In [21]:
ref_df = get_ref_group_df(ref_raw_df)
assert len(ref_df) == 200

## group train tmp df

In [22]:
def get_group_tmp_df(data_df, ref_df):
    group_df = data_df.groupby('business_id').agg({
        'tokens': lambda tokens_list: list(tokens_list),
        'stars': lambda stars_list: list(stars_list)
    })
    group_df = group_df.reset_index()
    ref_business_ids = ref_df['business_id'].values
    n_pre = len(group_df)
    group_df = group_df[group_df['business_id'].apply(lambda business_id: business_id not in ref_business_ids)]
    n_post = len(group_df)
    print('filtered %i business ids in references from train datasets' % (n_pre-n_post))
    return group_df

In [23]:
train_tmp_df = get_group_tmp_df(train_raw_df, ref_df)

filtered 166 business ids in references from train datasets


## batch train df

In [69]:
%%time
# Each item will appear 12 times
# 185496
# CPU times: user 34.6 s, sys: 1.8 s, total: 36.4 s
# Wall time: 35 s
train_df = get_group_df(train_tmp_df, n_reviews=config.n_reviews, filter_sent_l=config.filter_sent_l, filter_doc_l=config.filter_doc_l, \
                        item_min_reviews=config.item_min_reviews, n_per_item=config.n_per_item)

Each item will appear 12 times
CPU times: user 13.9 s, sys: 408 ms, total: 14.3 s
Wall time: 14.3 s


In [70]:
print(len(train_df))

185496


## split ref df into val & test df

In [37]:
test_indices = [111,  74, 106,  45, 143, 147,  56, 150, 184,  38,  61, 125, 116,
                             58, 159, 182,  86,   2,  22, 126,  55,  20, 161, 118, 141,  57,
                            123,  68, 164,  14, 122,  64,  53,  85, 135,  79, 163, 198, 109,
                            110,  25, 115, 113, 114,  78,  94, 151,  88, 162, 176,  66, 136,
                             62, 137, 158, 148, 171, 145,  52,   1,  82,   5, 173, 124, 190,
                            129, 185,  67, 107,   3, 193, 132,  69,  31,  41,  11, 108, 167,
                             96,  12, 139,  90,  23,  95,  21,   7,  54, 174,  65,  47, 194,
                            181, 153, 199, 121, 165,  80,  44, 188,  36]

In [38]:
test_df = ref_df.iloc[test_indices]
val_df = ref_df.drop(test_df.index)
assert len(set(test_df.index) & set(val_df.index)) == 0

# build token idxs for language modeling 

In [39]:
%%time
words_list = train_df['tokens'].apply(lambda tokens: [token for line in tokens for token in line])
word_tf_dict = sorted(Counter([word for words in words_list for word in words]).items(), key=lambda x: x[1])

CPU times: user 21.8 s, sys: 1.93 s, total: 23.7 s
Wall time: 23.7 s


In [50]:
special_words = [PAD, UNK, BOS, EOS]
lm_words = special_words + filter_words(word_tf_dict, min_tf=config.min_tf) # large, usual
idx_to_word = {idx: word for idx, word in enumerate(lm_words)}
word_to_idx = {word: idx for idx, word in idx_to_word.items()}
len(lm_words) # 32956

32956

In [43]:
%%time
train_df['token_idxs'] = apply_parallel(train_df['tokens'], num_split=64, map_func=apply_token_idxs)
val_df['token_idxs'] = apply_parallel(val_df['tokens'], num_split=64, map_func=apply_token_idxs)
test_df['token_idxs'] = apply_token_idxs(test_df['tokens'])

CPU times: user 39.9 s, sys: 56.8 s, total: 1min 36s
Wall time: 1min 38s


/home/isonuma/.pyenv/versions/anaconda3-5.3.1/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# write out

In [44]:
def get_save_df(data_df, summary=False):
    save_df = data_df[['business_id', 'doc_l', 'sent_l', 'max_sent_l', 'token_idxs', 'tokens']]
    if summary: save_df = data_df[['business_id', 'doc_l', 'sent_l', 'max_sent_l', 'tokens', 'token_idxs', \
                                                               'text', 'summary', 'summary_tokens', 'summary_doc_l', 'summary_max_sent_l']]
    return save_df

In [45]:
# (174794, 100, 100)
train_save_df = get_save_df(train_df)
val_save_df = get_save_df(val_df, summary=True)
test_save_df = get_save_df(test_df, summary=True)
len(train_save_df), len(val_save_df), len(test_save_df)

(185496, 100, 100)

In [46]:
config.output_path, len(word_to_idx)

('data/yelp/yelp_recursum_df.pkl', 32956)

In [59]:
print('saving preprocessed instances...')
cPickle.dump((train_save_df, val_save_df, test_save_df, word_to_idx, idx_to_word, None),open(config.output_path,'wb'))

saving preprocessed instances...
